# Clase 4 - Minimax y maximin
## Investigación Operativa 2C 2025 - Departamento de Matemática - FCEN-UBA

### Inicialización

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
!conda install -q pyscipopt

In [1]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

### Anuncios publicitarios

Resolver el modelo para decidir cuántos anuncios comprar en cada medio:

$$
    \begin{array}{crl}
        \max: & w &  \\
        \text{s.a.:} & 35x_1+29x_2+22x_3+27x_4 \leq & 850\\
         & x_1\leq &20\\
         & x_2\leq &15 \\
         & x_3\leq &25 \\
         & x_4\leq &15 \\
         & w \leq & 50x_1+31x_2+20x_3+26x_4 \\
         & w \leq & 32x_1+41x_2+15x_3+19x_4 \\
         & w \leq & 60x_1+38x_2+22x_3+25x_4 \\
         & x_1+x_2-x_3-x_4 \leq & 10 \\
         & x_1+x_2-x_3-x_4 \geq & -10 \\
         & x_3 \geq & 0.05(x_1+x_2+x_3+x_4) \\
         & w & \in\mathbb{R} \\
         & x_j & \in\mathbb{Z}_{\geq 0}\;\forall j
    \end{array}
$$


In [2]:
# Precio (en miles) por anuncio en cada medio
c = np.array([35, 29, 22, 27])

# Matriz con las proyecciones: cada columna corresponde a un medio, cada fila a una proyeccion
P = np.array([
    [50, 31, 20, 26],
    [32, 41, 15, 19],
    [60, 38, 22, 25]
])

# Vector con la cota superior de anuncios para cada medio
u = np.array([20, 15, 25, 15])

In [4]:
model = Model('Clase4')

X = np.empty(len(c), dtype = 'object')

medium_range = range(len(c))

for i in medium_range:
    X[i] = model.addVar(f'X_{i}', vtype='I')

W = model.addVar(f'W', vtype='C')

model.setObjective(W, sense='maximize')

model.addCons(sum(c[i] * X[i] for i in medium_range) <= 850)
for i in medium_range:
    model.addCons(X[i] <= u[i])

for j in range(len(P)):
    model.addCons(sum(P[j, i] * X[i] for i in medium_range) >= W)

model.addCons(X[0] + X[1] - X[2] - X[3] <= 10)
model.addCons(X[0] + X[1] - X[2] - X[3] >= -10)
model.addCons(X[0] + X[1] + X[2] + X[3] * 0.05 >= X[2])

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/clasesPracticas/clase4-my.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       0 del vars, 5 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 5 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 0 del vars, 6 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 6 deleted constraints, 0 added constraints, 10 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 5 variables (0 bin, 4 int, 1 impl, 0 cont) and 5 constraints
      5 constraints of type <linear>
trans